In [ ]:
import hmac, hashlib, base64, time, urllib.parse, urllib.request, json, urllib.error

IKEY = "DI23XH6SV1A6V4UY8BXF"
SKEY = "eAMP5oAJIuTGbT9yOg8gaH30nWzkK5pY7Nr1ZXxe"
HOST = "api-782a10d3.duosecurity.com"
USERNAME = "testuser"

VALID_SECS = 1800
COUNT = 1

DEBUG = False  # set True to print raw responses

def _require(name, val):
    if not val:
        print(f"Missing required setting: {name}. Set env var {name} in a prior cell.", file=sys.stderr)
        sys.exit(2)
for _n, _v in [("DUO_IKEY", IKEY), ("DUO_SKEY", SKEY), ("DUO_HOST", HOST)]:
    _require(_n, _v)

# ========= Helpers =========
def _canonical_query(params: dict) -> str:
    if not params:
        return ""
    return "&".join(f"{k}={urllib.parse.quote(str(params[k]), safe='~')}" for k in sorted(params))

def sign_request(method: str, host: str, path: str, params: dict, skey: str, ikey: str):
    date = time.strftime('%a, %d %b %Y %H:%M:%S -0000', time.gmtime())
    canon = "\n".join([date, method.upper(), host.lower(), path, _canonical_query(params)])
    sig = hmac.new(skey.encode(), canon.encode(), hashlib.sha1).hexdigest()
    auth = "Basic " + base64.b64encode(f"{ikey}:{sig}".encode()).decode()
    return date, auth

def duo_request(method: str, path: str, params: dict):
    date, auth = sign_request(method, HOST, path, params or {}, SKEY, IKEY)
    headers = {"Date": date, "Authorization": auth}
    url = f"https://{HOST}{path}"
    data = None

    if method.upper() == "GET":
        q = _canonical_query(params)
        if q: url += "?" + q
    else:
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        data = _canonical_query(params).encode()

    req = urllib.request.Request(url, data=data, headers=headers, method=method.upper())
    try:
        with urllib.request.urlopen(req, timeout=15) as r:
            raw = r.read().decode()
            if DEBUG: print("RAW OK:", raw)
            try:
                obj = json.loads(raw)
            except Exception:
                obj = raw  # non-JSON success (rare)
    except urllib.error.HTTPError as e:
        body = e.read().decode(errors="ignore")
        if DEBUG: print(f"RAW ERROR ({e.code}):", body)
        msg = None
        try:
            parsed = json.loads(body)
            if isinstance(parsed, dict):
                msg = parsed.get("message_detail") or parsed.get("message")
            else:
                msg = str(parsed)
        except Exception:
            pass
        if not msg:
            msg = body or f"{e.code} {e.reason}"
        raise RuntimeError(f"Duo HTTPError {e.code}: {msg}") from None
    except urllib.error.URLError as e:
        raise RuntimeError(f"Duo URLError: {e.reason}") from None

    if isinstance(obj, dict) and obj.get("stat") == "FAIL":
        msg = obj.get("message_detail") or obj.get("message") or "Unknown Duo error"
        raise RuntimeError(f"Duo API error: {msg}")

    return obj

# ========= Normalizers =========
def as_users(resp):
    """Return a list of user dicts regardless of Duo response shape."""
    if isinstance(resp, dict):
        r = resp.get("response", resp.get("users"))
        if isinstance(r, list): return r
        if isinstance(r, dict) and "users" in r: return r["users"]
    if isinstance(resp, list):
        return resp
    return []

def as_codes(resp):
    """
    Normalize bypass code responses into a list of dicts: [{"code": "..."}].
    Handles:
      - {"stat":"OK","response":{"bypass_codes":[{"code":"..."}]}}
      - {"response":{"bypass_codes":["123456"]}}
      - ["123456","234567"]
      - [{"code":"..."}]
      - [123456]  (ints)
    """
    # Extract the payload list first
    if isinstance(resp, dict):
        r = resp.get("response", resp)
        if isinstance(r, dict) and "bypass_codes" in r:
            payload = r["bypass_codes"]
        elif isinstance(r, list):
            payload = r
        else:
            payload = []
    elif isinstance(resp, list):
        payload = resp
    else:
        payload = []

    # Normalize each element into {"code": "<str>"}
    norm = []
    for item in payload:
        if isinstance(item, dict) and "code" in item:
            norm.append({"code": str(item["code"]),
                         "valid_secs": item.get("valid_secs"),
                         "remaining_uses": item.get("remaining_uses")})
        elif isinstance(item, str):
            norm.append({"code": item})
        elif isinstance(item, int):
            norm.append({"code": str(item)})
        else:
            # unknown element type; keep a best-effort repr
            norm.append({"code": str(item)})
    return norm

# ========= High-level =========
def get_user_id(username: str):
    resp = duo_request("GET", "/admin/v1/users", {"username": username})
    users = as_users(resp)
    return users[0]["user_id"] if users else None

def create_bypass(user_id: str, valid_secs: int = 1800, count: int = 1):
    resp = duo_request(
        "POST",
        f"/admin/v1/users/{user_id}/bypass_codes",
        {"count": int(count), "valid_secs": int(valid_secs)},
    )
    return as_codes(resp)

# ========= Run =========
try:
    uid = get_user_id(USERNAME)
    if not uid:
        raise SystemExit(f"User '{USERNAME}' not found in Duo.")

    codes = create_bypass(uid, VALID_SECS, COUNT)
    if not codes:
        raise SystemExit("No bypass code returned. Ensure Admin API has 'Grant/Manage Bypass Codes' permission.")

    first = codes[0]
    code = first["code"]  # always present after normalization
    print("User ID:", uid)
    print("Bypass code:", code)         # avoid printing full code in real logs
    print("Masked:", "****" + code[-4:] if len(code) >= 4 else code)
    print("Valid (secs):", first.get("valid_secs"))
    print("Remaining uses:", first.get("remaining_uses"))

except Exception as ex:
    print("Error:", ex)

User ID: DU2OA1ACC0OQV3KC3HJN
Bypass code: 377255273
Masked: ****5273
Valid (secs): None
Remaining uses: None
